# Provenance & License

**Source file:** `https://github.com/Yvictor/TradingGym/blob/master/README.md`   
**Original author:** Yvictor
**License:** MIT License (see upstream repository)

**Adapted by:** Xiangyu Ji

**Changes:**
- 

**Link to source:**  
https://github.com/Yvictor/TradingGym/blob/master/README.md

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import trading_env
import os

### Data Processing:

In [34]:
import numpy as np
import pandas as pd

def ohlcv_to_tradinggym(df_ohlcv: pd.DataFrame) -> pd.DataFrame:
    df = df_ohlcv.copy()

    df["datetime"] = pd.to_datetime(df["date"], utc=False)
    df = df.sort_values("datetime").dropna(subset=["close", "volume"])

    df["Price"]  = df["close"].astype(float)
    df["Volume"] = df["volume"].astype(float)

    spread = df["Price"] * 0.0001
    df["Ask_price"] = (df["Price"] + spread/2).astype(float)
    df["Bid_price"] = (df["Price"] - spread/2).astype(float)

    updown = np.sign(df["Price"].diff()).fillna(0.0)
    df["Updown"] = updown.astype(float)
    df["Updown_Cum"] = df["Updown"].cumsum()

    ask_ratio = np.where(df["Updown"] > 0, 0.7, np.where(df["Updown"] < 0, 0.3, 0.5))
    df["Ask_deal_vol"] = (df["Volume"] * ask_ratio).astype(float)
    df["Bid_deal_vol"] = (df["Volume"] - df["Ask_deal_vol"]).astype(float)

    df["Bid/Ask_deal"] = (df["Bid_deal_vol"] - df["Ask_deal_vol"]) / (
        (df["Bid_deal_vol"] + df["Ask_deal_vol"]).replace(0, np.nan)
    )
    df["Bid/Ask_deal"] = df["Bid/Ask_deal"].fillna(0.0)

    df["serial_number"] = np.arange(len(df), dtype=int)
    df["Comm"] = "CUSTOM"

    cols = [
        "serial_number","datetime","Comm","Price","Volume",
        "Ask_price","Bid_price","Ask_deal_vol","Bid_deal_vol",
        "Bid/Ask_deal","Updown","Updown_Cum"
    ]
    return df[cols]

In [35]:
train = pd.read_csv('../data/train_data.csv')
df = ohlcv_to_tradinggym(train)
df

,serial_number,datetime,Comm,Price,Volume,Ask_price,Bid_price,Ask_deal_vol,Bid_deal_vol,Bid/Ask_deal,Updown,Updown_Cum
0,0,2009-01-02,CUSTOM,2.724325,746015200.0,2.724462,2.724189,373007600.0,373007600.0,0.0,0.0,0.0
28,1,2009-01-02,CUSTOM,13.401936,48164400.0,13.402606,13.401265,33715080.0,14449320.0,-0.4,1.0,1.0
27,2,2009-01-02,CUSTOM,15.310209,5228648.0,15.310975,15.309444,3660053.6,1568594.4,-0.4,1.0,2.0
26,3,2009-01-02,CUSTOM,13.479150,14848985.0,13.479824,13.478476,4454695.5,10394289.5,0.4,-1.0,1.0
25,4,2009-01-02,CUSTOM,11.855903,13199200.0,11.856495,11.855310,3959760.0,9239440.0,0.4,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
94889,94912,2021-12-31,CUSTOM,199.557861,1668400.0,199.567839,199.547883,1167880.0,500520.0,-0.4,1.0,1152.0
94888,94913,2021-12-31,CUSTOM,174.090179,64062300.0,174.098884,174.081475,19218690.0,44843610.0,0.4,-1.0,1151.0
94915,94914,2021-12-31,CUSTOM,42.992260,3406668.0,42.994410,42.990110,1022000.4,2384667.6,0.4,-1.0,1150.0
94901,94915,2021-12-31,CUSTOM,47.764515,16319000.0,47.766903,47.762127,11423300.0,4895700.0,-0.4,1.0,1151.0


### Create Env:

In [36]:
env = trading_env.make(env_id='backtest_v1', obs_data_len=10, step_len=1,
                       df=df, fee=0.1, max_position=5, deal_col_name='Price', 
                       feature_names=['Price'])


[2025-10-31 16:23:39,594] Making new env: backtest_v1


### Train Env:

In [37]:
class RandomAgent:
    def __init__(self):
        pass
        
    def choice_action(self, state):
        return np.random.randint(3)


agent = RandomAgent()

transactions = []
while not env.backtest_done:
    state = env.reset()
    done = False
    while not done:
        state, reward, done, info = env.step(agent.choice_action(state))
        print(reward)
        #env.render()
        if done:
            transactions.append(info)
            break
transaction = pd.concat(transactions)
os.makedirs('../results', exist_ok=True)
transaction.to_csv("../results/Trading_gym.csv")


realized = env.info["reward_fluctuant"] * (env.info["reward_makereal"] != 0).astype(float)
initial_cash   = 100_000.0
contract_size  = 1.0

equity_curve = initial_cash + (realized.fillna(0.0) * contract_size).cumsum()
start_money  = equity_curve.iloc[0]
end_money    = equity_curve.iloc[-1]

print(f"Start: {start_money:.2f}")
print(f"End:   {end_money:.2f}")
print(f"Return: {(end_money/start_money - 1)*100:.2f}%")

print("Full transaction saved to Trading_gym.csv")

0.0
0.0
0.0
0.0
0.0
17.216007137298583
17.216007137298583
17.216007137298583
17.216007137298583
19.52735786437988
19.52735786437988
32.905911636352535
32.905911636352535
42.374734497070314
42.374734497070314
25.158727359771728
25.158727359771728
2.2249876022338952
2.2249876022338952
-0.08636312484740571
-0.08636312484740571
-13.464916896820062
-13.464916896820062
-22.933739757537836
-12.27094585895538
-12.27094585895538
-12.27094585895538
11.595016676187512
11.595016676187512
11.595016676187512
11.595016676187512
-6.7098883345723195
-6.7098883345723195
-6.7098883345723195
-17.372682233154777
-17.372682233154777
-17.372682233154777
-18.304905010759832
-25.615982592105865
-25.615982592105865
-25.615982592105865
9.322014289200302
9.322014289200302
9.322014289200302
16.408270920097824
16.408270920097824
17.093870806419847
17.093870806419844
34.36308070160865
34.36308070160865
34.36308070160865
17.729988831062315
30.871773994270317
30.871773994270317
23.785517363372794
23.785517363372794
23